# 2. Régression linéaire

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats

In [3]:
df = pd.read_csv("data/data.csv", header=0)

## Obtention des données

In [ ]:
col_academic_performance = "AcademicPerformanceLastorCurrentAcademicYear"
col_screen_time = "ScreentimeHoursDayMobileTablet"
col_outdoor_time = "OutdoorplayingHoursDay"
col_age = "Ageinyears"
academic_performance = (df[col_academic_performance] - df[col_academic_performance].mean()) / df[col_academic_performance].std()
screen_time = (df[col_screen_time] - df[col_screen_time].mean()) / df[col_screen_time].std()
outdoor_time = (df[col_outdoor_time] - df[col_outdoor_time].mean()) / df[col_outdoor_time].std()
age = (df[col_age] - df[col_age].mean()) / df[col_age].std()
top_percentile = 0.05
upper_threshold = academic_performance.quantile(1 - top_percentile)
lower_threshold = academic_performance.quantile(top_percentile)
mask = (academic_performance < upper_threshold) & (academic_performance > lower_threshold)
academic_performance_corrected = academic_performance[mask]
screen_time_corrected = screen_time[mask]
outdoor_time_corrected = outdoor_time[mask]
age_corrected = age[mask]

In [21]:
import numpy as np

In [46]:
filter = pd.DataFrame({'x': screen_time_corrected, 'y':academic_performance_corrected})
filter = filter.dropna(subset=['x','y'])
valS = filter['x'].tolist()
valA = filter['y'].tolist()
avgS = np.average(valS)
avgA = np.average(valA)

## Calcul de la formule linéaire

On cherche un résultat sous la forme:
$$
A = a \cdot S + b
$$
Avec $A$ étant le score académique et $S$ le temps d'ecran.

Les formules pour trouver $a$ et $b$ sont les suivantes:
$$a = \frac{\sum_{i=1}^{n} (S_i - \overline{S})(A_i - \overline{A})}{\sum_{i=1}^{n} (S_i - \overline{A})^2}$$
et
$$b = \overline{A} - a \cdot \overline{S}$$

In [47]:
aup = 0
adown = 0
for i in range(len(valS)):
    aup += (valS[i]-avgS)*(valA[i]-avgA)
    adown += (valS[i]-avgS)**2

a = aup/adown
b = avgA - a*avgS
print(a, b)

-0.2809476365110569 -0.01314831394346865


## Calcul du R²

pour calculer la pertinence de notre régression on obtiens le R² avec :
$$
R² = 1 - \frac {A_i - \overline{A}} {A_i - a \cdot S_i + b} 
$$

In [48]:
total = 0
res = 0
for i in range(len(valS)):
    distance = np.abs(avgA - a*valS[i]+b)
    total += (valA[i] - avgA)**2
    res += (valA[i] - distance)**2

RR = 1 - (total / res)

print(RR)

0.11959812831501526


Le R² va de 0 à 1 et mesure la qualité des prédictions sur les performances académiques réalisées en utilisant le temps d'écran. Une valeur de $0.12$ indique que le temps d'écran permet de prédire avec peu de précision les résultats académiques. Ces résultats montrent un lien certe présent mais très faible entre nos deux métriques, on peut en conclure qu'en moyenne les performances académiques ne devraient pas être prédites avec le temps d'écran car le lien entre les deux métriques est trop faible.

## Calcul du R² pour d'autres paramètres

### Temps passé à joué dehors

In [49]:
filter = pd.DataFrame({'x': outdoor_time_corrected, 'y':academic_performance_corrected})
filter = filter.dropna(subset=['x','y'])
valO = filter['x'].tolist()
valA = filter['y'].tolist()
avgO = np.average(valO)
avgA = np.average(valA)

aup = 0
adown = 0
for i in range(len(valO)):
    aup += (valO[i]-avgO)*(valA[i]-avgA)
    adown += (valO[i]-avgO)**2
a = aup/adown
b = avgA - a*avgO

total = 0
res = 0
for i in range(len(valO)):
    distance = np.abs(avgA - a*valO[i]+b)
    total += (valA[i] - avgA)**2
    res += (valA[i] - distance)**2

RR = 1 - (total / res)

print(RR)

0.10847299060576643


$R² = 0.11$ , comme pour le temps d'écran, le temps passé à jouer dehors a un lien faible avec les résultats académiques. On note une différence de $0.01$ entre ce R² et le R² précédent, cette différence est trop mineure pour justifier l'utilisation de l'un ou l'autre.

### Age

In [52]:
filter = pd.DataFrame({'x': age_corrected, 'y':academic_performance_corrected})
filter = filter.dropna(subset=['x','y'])
valB = filter['x'].tolist()
valA = filter['y'].tolist()
avgB = np.average(valB)
avgA = np.average(valA)

aup = 0
adown = 0
for i in range(len(valB)):
    aup += (valB[i]-avgB)*(valA[i]-avgA)
    adown += (valB[i]-avgB)**2
a = aup/adown
b = avgA - a*avgB

total = 0
res = 0
for i in range(len(valB)):
    distance = np.abs(avgA - a*valB[i]+b)
    total += (valA[i] - avgA)**2
    res += (valA[i] - distance)**2

RR = 1 - (total / res)

print(RR)

0.0060876292812857


$R² = 0.006$ , cette fois-ci, les résultats montrent qu'il n'y a pratiquement aucun lien entre l'âge et les performances académiques.

### Conclusion sur nos données


Les tests réalisés sur nos données montrent que le temps passé à jouer, que ce soit sur les écrans ou dehors, n'a que très peu d'impact sur les performances académiques. Une métrique qui aurait pu avoir un lien plus fort serait le temps passé à réviser. Bien que nos données ne possèdent pas ces informations, on imagine que la corrélation serait beaucoup plus importante.

La métrique de l'âge, quant à elle, ne semble avoir aucun lien sur les résultats. Cette métrique seule ne permet pas d'atteindre une conclusion précise, mais on peut imaginer que les difficultés du parcours académique augmentent autant que le niveau des élèves alors qu'ils grandissent.

On conclut donc que même si le temps passé à jouer a un impact sur les performances académiques, d'autres variables impactent les résultats finaux, il ne faut donc pas chercher à prédire les résultats académiques de son enfant par son temps passé à jouer.